# Atención primaria España

In [1]:
import pandas as pd
import os
import json

datos [porta estadístico](https://pestadistico.inteligenciadegestion.mscbs.es/publicoSNS/S/base-de-datos-de-clinicos-de-atencion-primaria-bdcap)

# consulta de datasets

In [2]:
psico_prim = pd.read_excel('../data/at_primaria/detalles_psico.xlsx')

In [3]:
psico_prim.columns = psico_prim.columns.str.replace(" ",'_')

In [4]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Grandes_Grupos,Problema_de_Salud,Casos
67842,2014,15-Navarra,Hombre,Sin Clasificar,00-14 años,P82 - Estrés postraumático,NaN
83978,2015,09-Cataluña,Hombre,?100.000 ? /año,65 y más años,P70 - Demencia,NaN
40329,2013,14-Murcia,Hombre,18.000-99.999 ? /año,15-34 años,P29 - Otros signos/síntomas psicológicos/mentales,NaN
24184,2013,04-Baleares,Mujer,?100.000 ? /año,15-34 años,P72 - Esquizofrenia,NaN
145873,2017,12-Galicia,Mujer,?100.000 ? /año,15-34 años,P13 - Encopresis,NaN


In [5]:
psico_prim.isna().sum()

Año                       0
CCAA                      0
Sexo                      0
Nivel_Renta               0
Grandes_Grupos            0
Problema_de_Salud         0
Casos                182993
dtype: int64

In [6]:
psico_prim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213840 entries, 0 to 213839
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Año                213840 non-null  int64  
 1   CCAA               213840 non-null  object 
 2   Sexo               213840 non-null  object 
 3   Nivel_Renta        213840 non-null  object 
 4   Grandes_Grupos     213840 non-null  object 
 5   Problema_de_Salud  213840 non-null  object 
 6   Casos              30847 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 11.4+ MB


In [7]:
psico_prim.columns

Index(['Año', 'CCAA', 'Sexo', 'Nivel_Renta', 'Grandes_Grupos',
       'Problema_de_Salud', 'Casos'],
      dtype='object')

## cambiamos algunos datos categóricos

In [8]:
psico_prim[["CodCA","CCAA"]] = psico_prim.CCAA.str.split("-",expand=True)

In [9]:
# usamos geojson de comunidades para poner el nombre a las comunidades

In [10]:
#diccionario
geo_json = r"../data/socioecon/comunidades.geojson"
with open (geo_json) as geo_file:
    comunidades = json.load(geo_file)

In [11]:
comunidades['features'][0]['properties']

{'cod_ccaa': '07',
 'noml_ccaa': 'COMUNIDAD DE CASTILLA Y LEON',
 'name': 'Castilla-Leon',
 'cartodb_id': 7,
 'created_at': '2014-09-30T00:00:00Z',
 'updated_at': '2014-12-25T02:07:41Z'}

In [12]:
cod_com = {}
for i in range(19):    
    cod_com[comunidades['features'][i]['properties']['cod_ccaa']] = comunidades['features'][i]['properties']['name']

In [13]:
psico_prim['CCAA'] = psico_prim.CodCA.map(cod_com)

In [14]:
list(psico_prim.Nivel_Renta.unique())

['Sin Clasificar',
 '?100.000 ? /año',
 '18.000-99.999 ? /año',
 '<18.000 ? /año',
 'Muy baja']

In [15]:
renta ={'Sin Clasificar':'desconocida',
 '?100.000 ? /año': '+ 100000',
 '18.000-99.999 ? /año': '18000-99999',
 '<18.000 ? /año':'- 18000',
 'Muy baja':'muy_baja'}

In [16]:
psico_prim.Nivel_Renta = psico_prim.Nivel_Renta.replace(renta)

In [17]:
psico_prim.Grandes_Grupos = psico_prim.Grandes_Grupos.str.extract(r'(\d+-*\d+)')
psico_prim.rename({"Grandes_Grupos":"Edad"},axis = 1,inplace = True)
psico_prim.Edad.replace('65','65 +', inplace = True)

In [18]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Problema_de_Salud,Casos,CodCA
60294,2014,Valencia,Mujer,desconocida,65 +,P15 - Abuso crónico del alcohol,NaN,10
167198,2018,Castilla-Leon,Mujer,muy_baja,15-34,P98 - Otras psicosis no especificadas,NaN,07
38763,2013,Madrid,Hombre,- 18000,15-34,P99 - Otros problemas psicológicos/mentales,993.0,13
91234,2015,Madrid,Hombre,desconocida,35-64,P25 - Problemas de las etapas de la vida en ad...,NaN,13
166675,2018,Castilla-Leon,Mujer,+ 100000,35-64,P04 - Sensación/sentimientos de irritabilidad/...,NaN,07


In [19]:
psico_prim.isna().sum()

Año                       0
CCAA                      0
Sexo                      0
Nivel_Renta               0
Edad                      0
Problema_de_Salud         0
Casos                182993
CodCA                     0
dtype: int64

In [20]:
psico_prim.Casos.fillna(0.,inplace = True)

In [21]:
psico_prim[["cod_salud","problema_salud"]]= psico_prim.Problema_de_Salud.str.split(" - ",expand=True)

In [22]:
psico_prim.drop('Problema_de_Salud',axis=1,inplace = True)

## equivalencias codigos diagnósticos con CEI10

In [23]:
trans_cod = {'P01': 'F40-49','P02': 'F40-49','P03': 'F40-49','P04': 'F40-49','P05':'F00-F09','P06': 'F30-39','P07': 'F30-39',
'P08': 'F30-39','P09': 'F30-39','P09-07': 'F30-39','P10': 'F90-F98','P11': 'F90-F98','P12': 'F90-F98','P13': 'F90-F98',
'P15': 'F10-F19','P16': 'F10-F19','P17': 'F10-F19','P18': 'F10-F19','P19': 'F10-F19','P20':'F00-F09','P22+23': 'F90-F98',
'P24': 'F80-89','P25': 'F60-69','P27': 'F99','P28': 'F70-79','P29': 'F99','P70':'F00-F09','P71': 'F20-29','P72': 'F20-29',
'P73': 'F20-29','P74': 'F40-49','P75': 'F40-49','P76': 'F30-39','P77': 'F30-39','P78': 'F40-49','P79': 'F40-49','P80': 'F60-69',
'P81': 'F90-F98','P82': 'F40-49','P82/02': 'F40-49','P85': 'F70-79','P86': 'F50-59','P98': 'F20-29','P99': 'F99',
}

In [24]:
psico_prim['cei10'] = psico_prim.cod_salud.map(trans_cod)

In [25]:
diagnos = pd.read_csv('../data/diagnos.csv')

In [27]:
diagnos.sample(2)

,code,diag,cap,title
20,F20,Esquizofrenia,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra..."
42,F51,Trastornos del sueño no orgánicos,F50-59,Síndromes del comportamiento asociados con alt...


In [28]:
diag_dic = {}
for i,r in diagnos.iterrows():
    diag_dic[r.cap] = r.title

In [30]:
psico_prim['title'] = psico_prim.cei10.map(diag_dic)

In [31]:
psico_prim.sample()

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10,title
48162,2014,Asturias,Mujer,18000-99999,65 +,965.0,03,P70,Demencia,F00-F09,"Trastornos mentales orgánicos, incluidos los t..."


## tipo columna casos

In [32]:
psico_prim.Casos =psico_prim.Casos.astype('int64')

In [33]:
psico_prim.sample(5)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10,title
196912,2019,Castilla-La Mancha,Hombre,18000-99999,65 +,0,08,P12,Enuresis,F90-F98,Trastornos emocionales y del comportamiento qu...
122145,2016,Navarra,Mujer,- 18000,15-34,318,15,P02,Estrés agudo,F40-49,"Trastornos neuróticos, trastornos relacionados..."
199118,2019,Cataluña,Hombre,desconocida,65 +,0,09,P19,Abuso de drogas,F10-F19,Trastornos mentales y de comportamiento debido...
36257,2013,Extremadura,Mujer,muy_baja,15-34,0,11,P02,Estrés agudo,F40-49,"Trastornos neuróticos, trastornos relacionados..."
102367,2016,Baleares,Hombre,+ 100000,15-34,0,04,P27,Miedo a una enfermedad mental,F99,Trastornos mentales sin especificar


In [35]:
psico_prim.to_csv('../data/psico_atprimaria.csv',index=False)